In [2]:

import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import cv2
import random 
import os, os.path
import matplotlib.pyplot as plt                                     
import imutils                                                      #contours

from torch.utils.data import Dataset, DataLoader, random_split
from torchinfo import summary                                       #network description 
from sympy import *                                                 #symbolic mathematics

LABELS_DICT = {
    '0' : 0,
    '1' : 1,
    '2' : 2,
    '3' : 3,
    '4' : 4,
    '5' : 5,
    '6' : 6,
    '7' : 7,
    '8' : 8,
    '9' : 9,
    'plus' : 10,
    'minus' : 11,
    'slash' : 12,
    'dot' : 13,
    'w' : 14,
    'x' : 15,
    'y' : 16,
    'z' : 17,
    }

INV_LABELS_DICS = {v: k for k, v in LABELS_DICT.items()}

In [30]:
"""
Student Network 
"""
class Student(nn.Module):
  def __init__(self, input_size, num_classes):
    super(Student, self).__init__()
    self.fc1 = nn.Linear(input_size, 30)
    self.fc2 = nn.Linear(30, num_classes)

  def forward(self, x):
    x = x.view(-1, 784)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    return x


#get previous network 
model = Student(input_size=784, num_classes=18)
model.load_state_dict(torch.load("model_new_student.pth"))

In [ ]:
"""
Save weights and bias
"""
for name, params in model.named_parameters():
  np.savetxt(name + ".txt", params.detach().numpy(), fmt="%s")

file = open("fc2.weight.txt")
output = open("fc2_weights.txt", "w")

for i in file:
    output.write(i.replace(" ", ","))

file.close()
output.close()


In [38]:
"""
Convert test images to tensor to get array data 
"""
#function to convert image to tensor 
def convert_to_tensor_flatten(image):
    #image = cv2.resize(image, (28, 28))     
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  
    _ , image = cv2.threshold(image_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)


    #transform to tensor 
    trans = transforms.Compose([transforms.ToTensor()])
    tensor_image = trans(image)

    #normalizing tensor
    mean, std = torch.mean(tensor_image), torch.std(tensor_image)
    normalize = transforms.Normalize(mean, std)
    tensor_image = normalize(tensor_image)

    return tensor_image.view(-1, 784)


array_image = cv2.imread("data/symbols/z-0000.png")
flatten_image = convert_to_tensor_flatten(array_image)


print(flatten_image)

    
    



tensor([[-0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -

In [31]:
z_array = [-0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,  2.5413,
          2.5413,  2.5413,  2.5413, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,  2.5413,  2.5413,  2.5413,
          2.5413,  2.5413,  2.5413,  2.5413,  2.5413,  2.5413,  2.5413,  2.5413,
          2.5413,  2.5413, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930,  2.5413,  2.5413,  2.5413, -0.3930, -0.3930, -0.3930,  2.5413,
         -0.3930, -0.3930, -0.3930, -0.3930,  2.5413,  2.5413,  2.5413, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
          2.5413,  2.5413,  2.5413, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930,  2.5413,  2.5413, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
          2.5413,  2.5413, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930,  2.5413,  2.5413,  2.5413, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,  2.5413,  2.5413,
          2.5413, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930,  2.5413,  2.5413,  2.5413, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930,  2.5413,  2.5413,  2.5413, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,  2.5413,
          2.5413,  2.5413, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930,  2.5413,  2.5413,  2.5413, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930,  2.5413,  2.5413,  2.5413,  2.5413,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,  2.5413,
          2.5413,  2.5413,  2.5413,  2.5413,  2.5413, -0.3930, -0.3930,  2.5413,
          2.5413,  2.5413, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
          2.5413,  2.5413,  2.5413,  2.5413,  2.5413,  2.5413,  2.5413,  2.5413,
          2.5413,  2.5413,  2.5413,  2.5413,  2.5413,  2.5413,  2.5413,  2.5413,
          2.5413,  2.5413,  2.5413,  2.5413, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930,  2.5413,  2.5413,  2.5413,  2.5413,
          2.5413,  2.5413,  2.5413,  2.5413, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930,  2.5413,  2.5413,  2.5413,  2.5413,  2.5413,
          2.5413,  2.5413,  2.5413,  2.5413,  2.5413,  2.5413,  2.5413, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
          2.5413,  2.5413,  2.5413, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930,
         -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930, -0.3930]

#print(len(dot_array))

784
